In [1]:
import asyncio, time, sys
import websockets, json
import numpy as np
import syncmetrics as syncm
from numpy_ringbuffer import RingBuffer

from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder
from pythonosc import udp_client
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

/Users/williamprimett1/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# OSC output config
local_ip = 'localhost'
local_port = 12000
output_address = '/results'
client = udp_client.SimpleUDPClient(local_ip, local_port)

#    Send to Processing/MAX/Etc...
def msg(values):
    client.send_message("/results", values[0])

#    Send to Processing/MAX/Etc...
def send_osc_bundle(values):
    bundle = osc_bundle_builder.OscBundleBuilder(osc_bundle_builder.IMMEDIATELY)
    msg = osc_message_builder.OscMessageBuilder(address=output_address)
    for arg_to_add in values:
        msg.add_arg(arg_to_add)
    bundle.add_content(msg.build())
    bundle = bundle.build()
    client.send(bundle)

def output_labeled(json_data):
    print(json_data)
    for key, value in json_data:
        output_address = "/result/" + label
        msg = osc_message_builder.OscMessageBuilder(
            address=output_address)
        arg_to_add = value
        msg.add_arg(arg_to_add)
        msg.build()
        self.client.send(msg)
#     bundle = osc_bundle_builder.OscBundleBuilder(
#         osc_bundle_builder.IMMEDIATELY)
    

In [7]:
class Session:
    device_data = json.dumps({})
    uri = 'ws://localhost:9001'
    ws_timeout = 5.0
    num_users = 2

    # to do processing in windoes 
    fs=1000
    buffer_W=150
    win_size=15*fs
    window_A = np.zeros(win_size)
    window_B = np.zeros(win_size)
    no_buffer=int(win_size/buffer_W)
    
    ema_buffers = RingBuffer(capacity=20, dtype=(float, num_users)) 
    feature_buffers = RingBuffer(capacity=no_buffer, dtype=(float, num_users))

    def calculate_features(self):
        device_data = json.loads(self.device_data)
        self.ema_buffers.appendleft([device_data["CH1"][0], device_data["CH2"][0]])
        sigA = (self.ema_buffers[0][0] - self.ema_buffers[0][-1])
        sigB = (self.ema_buffers[1][0] - self.ema_buffers[1][-1])
#         self.feature_buffers.appendleft([sigA, sigB])

        self.feature_buffers.appendleft([float(device_data["CH1"][0]), float(device_data["CH2"][0])])
        if self.feature_buffers.shape[0] > self.no_buffer-1:
            buff_A = np.array([i[0] for i in np.array(self.feature_buffers)])
            buff_B = np.array([i[1] for i in np.array(self.feature_buffers)])
            result_1=syncm.correlation_coeff(buff_A, buff_B).flat[0]/10
            result_2=syncm.cos_similarity(buff_A, buff_B).flat[0]
            result_3=syncm.correlation_coeff(buff_A, buff_B).flat[0]
        
            #add results here
            results = [result_1]
            print(result_1)
            msg(results)

    async def ServerBIT_recieve(self, uri):
        async with websockets.connect(uri) as websocket:
            while True:
                self.device_data = await websocket.recv()
                self.calculate_features()
                await asyncio.sleep(0.0)

    def run(self):
        uri = self.uri
        print("Serving on {}".format(uri))
        try:
            asyncio.get_event_loop().run_until_complete(
                self.ServerBIT_recieve(uri))
        except Exception as e:
            print(e)
            pass
        finally:
            print("no connection, try again")
            exit

In [8]:
session = Session()
session.run()

Serving on ws://localhost:9001
0.757929722857501
0.8422700012317305
0.8819955814165793
0.9055651355367341
0.9210596615601632
0.9318740328918452
0.9399026361884781
0.9460815370308147
0.9509649149009989
0.9549750842265832
0.9583219958690087
0.9611305853593429
0.9635320356597533
0.9655424302088762
0.9673355640172682
0.968882960808791
0.9702884268958231
0.9715355626505152
0.9725805764853573
0.9735694266456709
0.9744513509834395
0.9752565841911279
0.9759829151130067
0.9766434944231025
0.9772382376082273
0.9777985807933538
0.978286825427462
0.9787509828609146
0.9791726678046679
0.9795609238010796
0.9798928648335761
0.9802083891600973
0.9804905337381034
0.9807511242684538
0.9809879581306458
0.9811843262310871
0.9813718517286232
0.981525040396598
0.9816700786641559
0.9817792371092727
0.9818787807962756
0.9819549697619875
0.9820174778930467
0.9820592952285551
0.98208764081379
0.9820988412849215
0.9820986712851933
0.9820773696504004
0.9820396473345443
0.9819919711363031
0.9819210724159546
0.9818

-0.0781059999290963
-0.07729470271980349
-0.07632127688012039
-0.07525023693991258
-0.07399640313420504
-0.07250747303944036
-0.07050641236500707
-0.06837516844711475
-0.06548414750428688
-0.06258575465428479
-0.05956209898778474
-0.05592911442508845
-0.05285758948521206
-0.04932428444654199
-0.04556704212408436
-0.042292719114880785
-0.039258554366720586
-0.03640360989234591
-0.03267774313816241
-0.03036858217896702
-0.028199398753582895
-0.0233254081617223
-0.02059479531001434
-0.018652205736243967
-0.016452154575554028
-0.01463099205023238
-0.012123188653820274
-0.010204221790851349
-0.007775236959021302
-0.0057503533649344175
-0.004409436362206654
-0.003117085313005482
-0.0022890213449086603
-0.002213021964127526
-0.0016150316083720472
-0.00157393988058459
-0.0028205472093250855
-0.004809902994655816
-0.0067227281099495936
-0.009604969344135864
-0.011948119873342848
-0.015514547299203102
-0.019875323911924654
-0.02476404464791363
-0.03060169026434148
-0.03586852230929812
-0.0403945

0.09705377699331875
0.09642772967837272
0.09567240659309154
0.09471090975984077
0.09354821470612305
0.0920263274331024
0.09016840737648299
0.08803084731650448
0.08497771144473479
0.08133490809362869
0.07651792378782078
0.07068847131339147
0.0639845880755269
0.057030873141347235
0.04895385376990505
0.04041009807253321
0.03150219537343582
0.023274770185017044
0.014283778509778983
0.0049938911667119674
-0.00426613945129753
-0.012902184094232026
-0.02108723097734707
-0.027816226263272504
-0.03459923178323931
-0.04084300790267918
-0.04468163697767124
-0.04868199884355397
-0.05140373175778014
-0.053698123662860975
-0.056474893678909566
-0.057990982350481515
-0.05923864999909527
-0.06047359881293328
-0.06083922558984058
-0.06112289915750315
-0.06237388059794311
-0.062498925234942736
-0.06365043045035264
-0.06316006434708035
-0.0639486674752758
-0.06492270558165877
-0.06575774102308678
-0.06620418647168766
-0.06677507677167709
-0.06566004794685815
-0.06593636131277666
-0.06538384295976549
-0.0

-0.04055649052642462
-0.04235945101170282
-0.043978008599506235
-0.04554308221111926
-0.04701551738135275
-0.04850183229272816
-0.04995870264613856
-0.051274441019586295
-0.05269319250832263
-0.054086442807853165
-0.055368490728553335
-0.05666548388416036
-0.057915756610711855
-0.05890390272370698
-0.05990077906731304
-0.06078581818799208
-0.06159308504234753
-0.06196006908548081
-0.06255854378217209
-0.0631113817660213
-0.06319926042477715
-0.06310208533496268
-0.06303797036974924
-0.06268470687277784
-0.06195326263254458
-0.06128633006637094
-0.060472601951880124
-0.059206482266901464
-0.05754323833564022
-0.055960661348661046
-0.05371345993281572
-0.05086869860845021
-0.047715729379157454
-0.04423358514732047
-0.04032389186653394
-0.03583485979375424
-0.03112822281168274
-0.026469703949167738
-0.021739242654615448
-0.017273022846748524
-0.013176804520343264
-0.008998040864678428
-0.005116062265390896
-0.0010467161794365002
0.0022510386412974938
0.005473661189771976
0.008787140853959

0.03814477961575333
0.04294122786270958
0.04732418736029585
0.0512174075342709
0.054852637099374824
0.05833558572424047
0.061543442510899646
0.06444479279518528
0.0671190747240527
0.06967894802290805
0.07193506787511528
0.07406155904955751
0.07610552843875815
0.0778791289450561
0.07951269477804783
0.0810514692403261
0.08248475730322777
0.08385031504287296
0.08497453394485818
0.08611025957634193
0.0871840980558724
0.08814273349658507
0.08898312094607864
0.08983665182070291
0.09058650405369036
0.09112841246224937
0.09178553081123263
0.09238994986282631
0.092988747544949
0.09350362844523552
0.09392678260251251
0.09437717075685466
0.09479814658907512
0.09517612179925625
0.09554295686615626
0.0958480410975852
0.09612645290425251
0.09631054844723545
0.09651887359134122
0.09670801537586714
0.09691948397178532
0.0970642877474987
0.09712999492912548
0.09719527532591664
0.09727046448363698
0.09729924783417579
0.09723350138956714
0.09711321021260855
0.09692147968907702
0.09668733622709098
0.09645

...

-0.01170092332657099
-0.005651587183393734
9.315763556912087e-05
0.005144013799866328
0.010840272488780419
0.01615505440452507
0.020085945000501614
0.024193965292983698
0.02913578846493714
0.033830107375848156
0.0380503582816735
0.042413951882138895
0.04622632981158688
0.05013494333766424
0.05423403621529936
0.05755793133594954
0.06084101043749986
0.06321233100652329
0.06570526023127646
0.06771310384914903
0.06947156899345873
0.07130022525203719
0.07269831291023318
0.07397529493222349
0.07498681515806274
0.07541988934020652
0.07550234973694918
0.075732641020228
0.07532045909095503
0.07466889876768523
0.07423425867888314
0.07361299729199162
0.07286113520326781
0.07216867385935448
0.07100590233609112
0.0706452667828836
0.06999312279347931
0.06962206459573433
0.06933131575701834
0.06815524666353767
0.06765329077850366
0.06624507531355944
0.06584425682170496
0.06584069236328964
0.06540205057275342
0.06541999189238658
0.0660404954367717
0.0667921370047494
0.06725240727669227
0.0676799197182

-0.008996133437718556
-0.006343655213584211
-0.004212696593337065
-0.0025097826903289202
-0.0011379295689747369
-0.0001242014251260156
0.0005093740228269007
0.0012237619012907708
0.00188750685965644
0.0028994261849581765
0.004154818839181191
0.005637709101861596
0.007829491282772275
0.010265619811294494
0.013115768538014647
0.015946448857866505
0.019507833821398694
0.022976413545268047
0.027034014573975418
0.03201898240635479
0.03715518996439003
0.04247729157089135
0.04791499573768589
0.05272859114955941
0.057643069410764594
0.06243034253638975
0.06675086519192422
0.0707799527290673
0.07485352028664971
0.07717238963094297
0.07905527626300424
0.08047111182433091
0.08126614161856822
0.08179237378474861
0.08214481805030735
0.08223839588791625
0.08223464384989011
0.08212947657233785
0.08190593830542223
0.0816071614920521
0.08127459228160236
0.08091872153428148
0.0804903097016004
0.08002705343330804
0.07951366117692762
0.07896139383336126
0.0783805117647673
0.07782572023857907
0.07733906236

-0.09572121566868295
-0.09557703081500055
-0.09551088046698582
-0.09532802091240519
-0.09514750614701702
-0.09507934644148913
-0.09500009721859062
-0.09489574129982307
-0.09471345394546063
-0.09459402099381692
-0.09439163924970645
-0.09427377364207651
-0.09420216654008208
-0.09399027716145582
-0.0939038370564533
-0.09367459535425295
-0.09339998770421887
-0.09327208704065229
-0.09307062527489895
-0.09348208423866634
-0.09319980454441251
-0.09304167683461737
-0.09264806240787749
-0.09245179361956529
-0.09199520773771057
-0.0918414498527614
-0.09178554428839311
-0.09140887315548726
-0.09110547769244906
-0.09063342264114445
-0.09038806673850382
-0.089973388815938
-0.08914289524212483
-0.08892972835812729
-0.08849505145866884
-0.08817581955656723
-0.08805887188800096
-0.087313542692697
-0.0867557443363848
-0.08609356720945636
-0.08568450951316853
-0.08478547893614205
-0.08438708813243885
-0.08408409595034044
-0.08395971462964313
-0.0829717837737223
-0.0826485432951182
-0.0820468045152071
-0

-0.03357055469588678
-0.03390516203215698
-0.03330444598475043
-0.03083257624792579
-0.029346572698272223
-0.029449788302615864
-0.029466769366496365
-0.030106455135114833
-0.030789686524081213
-0.030017143946127384
-0.030621763446442535
-0.030185516651378697
-0.030599941589194202
-0.02905702514054324
-0.02872170294519157
-0.029269776778807505
-0.0268183859685542
-0.026490330604576718
-0.02585729213120926
-0.025415228414473284
-0.024896499968086624
-0.02553225537196329
-0.02537263030794596
-0.021259482567359416
-0.02100015777472109
-0.019559283041631067
-0.018406504592373474
-0.01479525548640731
-0.011869096569988604
-0.010365949197021428
-0.009916773310391123
-0.009390677438421298
-0.006892155575983036
-0.006369544845778103
-0.00717159610646983
-0.003389501843612229
-0.00034443084945024003
0.0008499209612390079
0.0002979865257498117
0.005316529028463218
0.005898118215328126
0.006809810362198833
0.00614338664175894
0.004608204613204432
0.002803298038735272
0.004865659657136425
0.007351

-0.03329134576868585
-0.034923355004786195
-0.03422102349490287
-0.03517691095683933
-0.03599858233061139
-0.03662252407928903
-0.035430680919061965
-0.03470247052172716
-0.03343762854945791
-0.03470981688801446
-0.034945852573533484
-0.051283552720376324
-0.051090665056271536
-0.05195976874992321
-0.051017831827744164
-0.04960356730751826
-0.04819814052847357
-0.050989975348515015
-0.049237705615058296
-0.050652018909632515
-0.05021789511383985
-0.04978833654860045
-0.04884738302746287
-0.04925218132687999
-0.049088856291299096
-0.047079906747443154
-0.04718663962708711
-0.04638242923628128
-0.04683628244089191
-0.04588927836385215
-0.047237182658799746
-0.050180488207965654
-0.05079479935721436
-0.049777348881686174
-0.04775684704297803
-0.04994601178311521
-0.05013002893467746
-0.05132669722101354
-0.052231949258425045
-0.05257845295359124
-0.0528140336311109
-0.05155488378246996
-0.052549268529452874
-0.05182914105118223
-0.05168024237976339
-0.052135011646807325
-0.051265679549327